## Instances
[Li & Lim Benchmark](https://www.sintef.no/projectweb/top/pdptw/li-lim-benchmark/)

In [105]:
# download instances
!sh instance.sh

mkdir: cannot create directory ‘Instance’: File exists
--2017-11-14 18:43:26--  https://www.sintef.no/contentassets/1338af68996841d3922bc8e87adc430c/pdp_100.zip
Resolving www.sintef.no (www.sintef.no)... 78.91.101.14
Connecting to www.sintef.no (www.sintef.no)|78.91.101.14|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 74121 (72K) [application/x-zip-compressed]
Saving to: ‘pdp_100.zip’

pdp_100.zip         100%[===================>]  72,38K   117KB/s    in 0,6s    

2017-11-14 18:43:28 (117 KB/s) - ‘pdp_100.zip’ saved [74121/74121]

Archive:  pdp_100.zip
  inflating: pdp_100/lc101.txt       
  inflating: pdp_100/lc102.txt       
  inflating: pdp_100/lc103.txt       
  inflating: pdp_100/lc104.txt       
  inflating: pdp_100/lc105.txt       
  inflating: pdp_100/lc106.txt       
  inflating: pdp_100/lc107.txt       
  inflating: pdp_100/lc108.txt       
  inflating: pdp_100/lc109.txt       
  inflating: pdp_100/lc201.txt       
  inflating: pdp_100/lc202.txt  

Connecting to www.sintef.no (www.sintef.no)|78.91.101.14|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 613 [text/plain]
Saving to: ‘lrc206.txt.1’

lrc206.txt.1        100%[===================>]     613  --.-KB/s    in 0s      

2017-11-14 18:43:42 (77,0 MB/s) - ‘lrc206.txt.1’ saved [613/613]

--2017-11-14 18:43:42--  https://www.sintef.no/contentassets/abb42d6b3f82453cb1908ed546b24fca/lrc107.txt
Resolving www.sintef.no (www.sintef.no)... 78.91.101.14
Connecting to www.sintef.no (www.sintef.no)|78.91.101.14|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 727 [text/plain]
Saving to: ‘lrc107.txt.1’

lrc107.txt.1        100%[===================>]     727  --.-KB/s    in 0s      

2017-11-14 18:43:43 (94,5 MB/s) - ‘lrc107.txt.1’ saved [727/727]

--2017-11-14 18:43:43--  https://www.sintef.no/contentassets/abb42d6b3f82453cb1908ed546b24fca/lrc207.txt
Resolving www.sintef.no (www.sintef.no)... 78.91.101.14
Connecting to www.sintef.no (www.s

In [107]:
!head -10 Instance/data/lrc101.txt

# Hybrid evolutive algorithm

In [54]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

In [101]:
class Problem:
    def __init__(self, filePath):
        with open(filePath, 'r') as file:
            self.N = int(file.readline().split()[0])
            X = []
            Y = []
            D = []
            I = []
            for lines in file.read().split('\n'):
                if lines:
                    i, x, y, _, _, _, _, _, d = lines.split()
                    I.append(int(i))
                    X.append(np.float(x))
                    Y.append(np.float(y))
                    D.append(int(d))
            X = np.array(X)
            Y = np.array(Y)
            self.dist = np.abs(X - Y.reshape(Y.size,1))
            self.G = nx.from_numpy_matrix(self.dist)
            self.request = dict(zip(I, D))

    def draw(self):
        nx.draw(self.G)

In [15]:
class Nucleosome:
    def __init__(self, sol=np.array([]), key=np.array([])):
        self.sol = self.reconstruct(sol, key)
        self.key = self.randkey(key)
        self.path = np.argsort(self.sol)[(self.sol==0).sum():] + 1
    
    def reconstruct(self, sol, key):
        if sol.any():
            return sol
        pos = 1
        sol = np.array(key, copy=True)
        for i in range(len(sol)):
            if sol[i]:
                sol[i] = pos
                pos += 1
        return sol.astype(int)
            
    def randkey(self, key):
        if key.size:
            return key
        return (self.sol != 0)*4*np.exp(-self.sol)/(1 + np.exp(-self.sol))**2    


In [104]:
class Chromosome:
    def __init__(self, nucList=np.array([]), gen=np.array([]), problem=None):
        self.N = problem.N
        if nucList.size:
            self.sol = amap(lambda x: x.sol, nucList)
            self.key = amap(lambda x: x.key, nucList)
            self.gen = self.key.flatten()
            self.path = amap(lambda x: x.path, nucList)
        else:
            self.gen = gen
            self.key = gen.reshape(self.N, int(len(gen)/self.N))
            self.sol = self.reconstruct()
            self.path = self.pathConstruction()
        self.fitness = None
        self.fit() 
        
    def reconstruct(self):
        pos = 1
        sol = np.array(self.key, copy=True)
        if self.N == 1:
            for i in range(len(sol)):
                if sol[i]:
                    sol[i] = pos
                    pos += 1
        else:
            for i in range(sol.shape[0]):
                pos = 1
                for j in range(sol.shape[1]):
                    if sol[i][j]:
                        sol[i][j] = pos
                        pos += 1
        return sol.astype(int)

    def pathConstruction(self):
        path = list(self.sol)
        for i in range(len(path)):
            path[i] = path[i].argsort()[(path[i]==0).sum():] + 1
        return np.array(path)
    
    def fit(self):
        if self.factibleQ():
            # TODO total-distance
        else:
            self.fitness = np.inf
    
    def factibleQ(self):
        # TODO
        
def amap(func, *args):
    args = np.broadcast(None, *args)
    res = np.array([func(*arg[1:]) for arg in args])
    shape = args.shape + res.shape[1:]
    return res.reshape(shape)